# BT-BERT Workflow
Bu notebook, BlueSense kozmetik öneri hattındaki tüm önemli adımları tek bir yerden çalıştırmak için hazırlandı. Aşağıdaki bölümler sırayla veri kümelerini normalleştirir, pseudo-label üretir, modeli değerlendirir ve son olarak hibrit öneri demoları üretir. Her bölümdeki kod hücresinin neden çalıştırıldığı ve hangi dosyaları etkilediği ayrıntılı olarak açıklanmıştır.


# 0. Hazırlık
Bu bölüm, notebook boyunca tekrar kullanılacak tüm dizin ve konfigürasyon değişkenlerini hazırlar. Kod hücresi proje kökünü, veri klasörlerini ve Dataset_Pipeline paketini `sys.path` içerisine alır; böylece sonraki hücreler modül importlarında sorun yaşamadan doğrudan bu yolları kullanabilir. Ayrıca `CONFIG_PATH`, `DATA_DIR`, `RAW_DATA_DIR` ve ham veri kaynağını temsil eden `DATASET_DIR` gibi kritik yolları gösterecek biçimde ekrana yazdırır.


In [1]:
from pathlib import Path
import subprocess
import sys

SEARCH_PATHS = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
for candidate in SEARCH_PATHS:
    if (candidate / 'bt_bert_model').exists() and (candidate / 'Dataset_Pipeline').exists():
        REPO_ROOT = candidate
        break
else:
    raise RuntimeError('Repository root could not be located relative to the notebook.')

PROJECT_ROOT = REPO_ROOT / 'bt_bert_model'
DATASET_PIPELINE_ROOT = REPO_ROOT / 'Dataset_Pipeline'
SERVICE_ROOT = REPO_ROOT / 'service'
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
DATASET_DIR = REPO_ROOT / 'Dataset'
for path in (REPO_ROOT, PROJECT_ROOT, DATASET_PIPELINE_ROOT, SERVICE_ROOT):
    if path.exists() and str(path) not in sys.path:
        sys.path.append(str(path))

CONFIG_PATH = PROJECT_ROOT / 'config.yaml'
print(f'Working directory     : {Path.cwd()}')
print(f'Repository root       : {REPO_ROOT}')
print(f'Dataset source dir    : {DATASET_DIR}')
print(f'Pipeline output dir   : {RAW_DATA_DIR}')


Working directory     : c:\Users\barut\OneDrive\Desktop\BlueSenseRecommendation\Recommendation-Systems\bt_bert_model\notebooks
Repository root       : c:\Users\barut\OneDrive\Desktop\BlueSenseRecommendation\Recommendation-Systems
Dataset source dir    : c:\Users\barut\OneDrive\Desktop\BlueSenseRecommendation\Recommendation-Systems\Dataset
Pipeline output dir   : c:\Users\barut\OneDrive\Desktop\BlueSenseRecommendation\Recommendation-Systems\bt_bert_model\data\raw


# 1. Dataset Pipeline
Bu adım ham EWG kategori CSV dosyalarını ingest eder, kolonları hizalar, ürün kimliklerini yeniden oluşturur ve normalize edilmiş ingredient listelerini çıkarır. `Dataset_Pipeline.data_pipeline.run_pipeline` fonksiyonu çalıştırılarak: (1) `Dataset/` klasöründeki tüm dosyalar okunur, (2) kategori ve ingredient değerleri normalize edilir, (3) çıktı olarak `bt_bert_model/data/raw/` altında `unified_products.csv`, `ingredient_normalisation_map.csv` ve `unique_ingredients.csv` dosyaları üretilir. Kod hücresi ayrıca yürütme süresini raporlar ve ürün tablosunun ilk satırlarını önizleyerek pipeline’ın doğru çalıştığını doğrular.


In [2]:
import importlib.util
from datetime import datetime

module_path = DATASET_PIPELINE_ROOT / 'data_pipeline.py'
if not module_path.exists():
    raise FileNotFoundError(f'Dataset pipeline module not found at {module_path}')
spec = importlib.util.spec_from_file_location('dataset_pipeline', module_path)
data_pipeline = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_pipeline)

pipeline_start = datetime.now()
pipeline_outputs = data_pipeline.run_pipeline(DATASET_DIR, RAW_DATA_DIR)
elapsed = datetime.now() - pipeline_start
print(f'Dataset pipeline tamamlandı: {elapsed}.')
print(f'Products table size: {pipeline_outputs.products.shape}')

preview_cols = ['product_id', 'name', 'category', 'ingredient_count']
available_cols = [col for col in preview_cols if col in pipeline_outputs.products.columns]
preview = pipeline_outputs.products.head()[available_cols]
preview


Dataset pipeline tamamlandı: 0:00:07.864910.
Products table size: (16556, 11)


,product_id,name,category,ingredient_count
0,0,Grace + Tonic Botanical Beauty Cleansing Mud (...,facial cleanser,24
1,1,Murad Hydration AHA/BHA Exfoliating Cleanser,facial cleanser,21
2,2,Purity One Step Mattifying Facial Cleanser,facial cleanser,22
3,3,"Every Man Jack, Skin. Volcanic Clay 2 in 1 Mas...",facial cleanser,10
4,4,Pond's Deep Cleanser Make Up Remover Cold Crea...,makeup remover,13


# 2. Data Labelling
Pipeline çıktılarının ardından bu adım pseudo-label üretim sürecini otomatikleştirir. Hücre, `src/data_prep.py` betiğini aynı Python süreç içinde çalıştırır; konfigürasyon olarak `bt_bert_model/config.yaml` kullanılır. İşlem sonucunda `bt_bert_model/data/` dizininde `labels.csv`, `train.csv`, `val.csv` ve `test.csv` dosyaları güncellenir. Kod her dosyanın varlığını kontrol edip durumunu ekrana yazdırır; böylece veri hazırlama aşamasının başarılı şekilde tamamlandığı doğrulanır.


In [3]:
# import os
# import subprocess
# import sys

# data_prep_cmd = [
#     sys.executable,
#     str(PROJECT_ROOT / 'src' / 'data_prep.py'),
#     '--config',
#     str(CONFIG_PATH),
# ]
# env = os.environ.copy()
# py_path_entries = [str(REPO_ROOT), str(PROJECT_ROOT), str(DATASET_PIPELINE_ROOT)]
# existing_py_path = env.get('PYTHONPATH', '')
# combined_py_path = os.pathsep.join([entry for entry in py_path_entries if entry] + ([existing_py_path] if existing_py_path else []))
# env['PYTHONPATH'] = combined_py_path
# print('Running:', ' '.join(str(part) for part in data_prep_cmd))
# subprocess.run(data_prep_cmd, check=True, env=env)

# required = [
#     DATA_DIR / 'labels.csv',
#     DATA_DIR / 'train.csv',
#     DATA_DIR / 'val.csv',
#     DATA_DIR / 'test.csv',
# ]
# for path in required:
#     status = 'OK' if path.exists() else 'MISSING'
#     print(f'{path.name}: {status}')


# 3. Label Özetini İnceleme
`label_summary.json` dosyası, her concern için pozitif/negatif örnek sayısını içerir. Bu hücre dosyayı okuyup okunabilir bir tabloya dönüştürerek label dağılımının dengeli olup olmadığını gözlemlemenizi sağlar. Eksik dosya durumunda kullanıcıyı uyarır ve bir önceki adımı tekrar çalıştırması gerektiğini belirtir.


In [4]:
import pandas as pd
import json

summary_path = Path("../data/label_summary.json")
if summary_path.exists():
    summary = json.loads(summary_path.read_text(encoding="utf-8"))
    display(pd.DataFrame(summary).T)
else:
    print("Run the data labelling step first.")

,positive,negative
acne,7887,8669
age,6863,9693
eyebag,1558,14998
moisture,14393,2163
oiliness,7841,8715
redness,8023,8533
wrinkle,7876,8680


# 4. Eğitim (Devre Dışı)
Bu notebook eğitim hücresini yalnızca referans amaçlı tutuyor; eğitim clustera taşındığı ve güncel model checkpointleri kullanıldığı için kod kısmı yorum satırına alınmıştır.


In [5]:
# Disable local training; use checkpoints copied from cluster.
from pathlib import Path
ckpt_dir = Path('../outputs/checkpoints')
ckpts = sorted(ckpt_dir.glob('*.pt')) if ckpt_dir.exists() else []
if ckpts:
    print(f'Found {len(ckpts)} checkpoints; skipping training.')
else:
    print('No checkpoints found. Copy from cluster or run a short local test:')
    print('  python ../src/train.py --config ../config.yaml --max_steps 10')


No checkpoints found. Copy from cluster or run a short local test:
  python ../src/train.py --config ../config.yaml --max_steps 10


# 5. Değerlendirme
Bu bölüm, kayıtlı checkpoint’ler üzerinden `src/evaluate.py` komutunu nasıl çalıştıracağınızı gösterir. Kod mevcut `outputs/checkpoints/` dizininde son kaydedilen modeli bulur, eğer bulunursa değerlendirme komutunu çıktısıyla birlikte çalıştırır ve sonuçların nereye yazıldığını bildirir. Checkpoint yoksa kullanıcıya önce eğitim ya da harici bir checkpoint yerleştirmesi gerektiğini söyler.


## 4b. GPU Eğitim Scripti (Devre Dışı)
Tüm eğitim süreçleri clustera taşındığından bu hücre de yorum satırına alınmıştır. Gerekirse `bt_bert_model/scripts/run_gpu_training.py` dosyasını doğrudan çalıştırabilirsiniz.


In [6]:
# from bt_bert_model.scripts.run_gpu_training import main as run_gpu_training_main

# try:
#     run_gpu_training_main(['--preset', 'debug'])
# except SystemExit as exc:
#     print(exc)


In [7]:
# from pathlib import Path
# import json
# import subprocess

# checkpoint_dir = Path("../outputs/checkpoints")
# checkpoints = sorted(checkpoint_dir.glob("*.pt"))
# print("Found checkpoints:")
# for ckpt in checkpoints:
#     print(ckpt.name)

# if checkpoints:
#     selected_checkpoint = checkpoints[-1]
#     metrics_output = Path("../outputs/eval_metrics.json")
#     cmd = ["python", "../src/evaluate.py", "--config", str(CONFIG_PATH), "--checkpoint", str(selected_checkpoint), "--split", "test", "--output", str(metrics_output)]
#     print('Running:', ' '.join(cmd))
#     subprocess.run(cmd, check=True)
#     print(json.loads(metrics_output.read_text(encoding='utf-8')))
# else:
#     print('Train the model first to generate checkpoints.')

# 6. Açıklanabilirlik
Modelin dikkat mekanizmasını analiz etmek için `src/explain.py` betiği kullanılır. Bu hücre, geçerli bir checkpoint bulunduğunda dikkat raporlarının (`outputs/attention_reports/`) nasıl üretileceğini adım adım gösterir. Hiçbir dosya oluşturulmamışsa kullanıcı uygun checkpoint’i temin etmesi için bilgilendirilir.


In [8]:
# from pathlib import Path
# import json
# import subprocess

# explain_output = Path("../outputs/attention_reports/explanations.json")
# explain_output.parent.mkdir(parents=True, exist_ok=True)

# if checkpoints:
#     selected_checkpoint = checkpoints[-1]
#     cmd = ["python", "../src/explain.py", "--config", str(CONFIG_PATH), "--checkpoint", str(selected_checkpoint), "--split", "val", "--sample", "50", "--output", str(explain_output)]
#     print('Running:', ' '.join(cmd))
#     subprocess.run(cmd, check=True)
#     explanations = json.loads(explain_output.read_text(encoding='utf-8'))
#     print(f"{len(explanations)} explanations written to {explain_output}")
# else:
#     print('No checkpoints available. Train the model first.')

# 7. Çıktıları İnceleme
Eğitim veya değerlendirme sonrası üretilen `bt_bert_outputs.csv` dosyası model tahminlerini içerir. Hücre, dosya mevcut değilse otomatik olarak `src/predict.py` betiğini çağırarak en güncel checkpoint ile tahmin üretir; dosya varsa ilk birkaç satırı görüntüleyerek model skorlarının incelemesini kolaylaştırır.


In [9]:
# import pandas as pd
# from pathlib import Path
# import subprocess

# output_path = Path('../outputs/bt_bert_outputs.csv')
# if not output_path.exists():
#     print('Creating prediction output using latest checkpoint...')
#     checkpoint_dir = Path('../outputs/checkpoints')
#     checkpoints = sorted(checkpoint_dir.glob('*.pt'))
#     if checkpoints:
#         selected_checkpoint = checkpoints[-1]
#         cmd = [
#             'python',
#             '../src/predict.py',
#             '--config',
#             str(CONFIG_PATH),
#             '--checkpoint',
#             str(selected_checkpoint),
#             '--split',
#             'test',
#             '--output',
#             str(output_path),
#         ]
#         print('Running:', ' '.join(cmd))
#         subprocess.run(cmd, check=True)
#     else:
#         print('No checkpoints available. Train the model first.')

# if output_path.exists():
#     df = pd.read_csv(output_path)
#     display(df.head())
# else:
#     print('No exported recommendation outputs yet.')

---
Bu notebook, veriyi ham halinden normalize edilmiş formata dönüştürmekten, pseudo-label üretimine, model çıktılarının doğrulanmasına ve hibrit tavsiye demosuna kadar uzanan uçtan uca bir çalışma akışı sunar. Her bölümün açıklamalarında hangi dosyaların okunduğu/yazıldığı ve komutların ne yaptığı detaylı biçimde yer almaktadır.


## 8. Recommendation Demo
Bu bölüm, Hybrid Concern deneylerinden elde edilen `product_concern_weights.csv` dosyasını kullanarak profil bazlı önerilerin nasıl üretileceğini gösterir. İlk hücre hibrit öneri sınıfını ve örnek kullanıcı profillerini yükler; ikinci hücre seçilen demo kullanıcılarının concern vektörlerini alır, `HybridRecommender` ile ürünleri skorlar ve her öneri için hibrit puanı ile model olasılıklarını satır satır ekrana yazar. Böylece gerçek bir kullanıcı metadatası geldiğinde servisin nasıl davranacağı notebook üzerinden görülür.


In [10]:
from service.hybrid_recommender import HybridRecommender, load_demo_profiles

weights_path = REPO_ROOT / 'Experiments' / 'Hybrid_Concern_Test' / 'product_concern_weights.csv'
print(f'Hibrit ağırlık dosyası: {weights_path}')
hybrid = HybridRecommender(weights_path=weights_path)
demo_profiles = load_demo_profiles()
sorted(demo_profiles.keys())[:5]  # profillerin bir kısmını göster


Hibrit ağırlık dosyası: c:\Users\barut\OneDrive\Desktop\BlueSenseRecommendation\Recommendation-Systems\Experiments\Hybrid_Concern_Test\product_concern_weights.csv


['focused_acne',
 'focused_age',
 'focused_eyebag',
 'focused_moisture',
 'focused_oiliness']